In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import f1_score
import time

In [7]:
# Load the dataset
data_path = '/Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/'
dataset = np.load(data_path + 'PCA_dataset1.npy', allow_pickle=True)[()]

x_train = dataset['train_stars_pca']
x_val = dataset['validation_stars_pca']
x_test = dataset['test_stars_pca']
y_train = dataset['train_C']
y_val = dataset['validation_C']
y_test = dataset['test_C']
SED_test = dataset['test_SEDs']


In [8]:
# Hyperparameters
PCA_components = 24
model_learning_rate = 0.1
N_epochs = 50
N_committee = 48

In [9]:
def SEDlisttoC(SED_list):
    sed_array = np.array(SED_list)
    return sed_array*0.5 + 1.5

def CtoSEDarray(c_values, variance):
    sed_classes = ((c_values - 1.25) // 0.5).astype(int)
    sed_classes = np.where((c_values < 1.25) | (c_values > 7.75), 20, sed_classes)
    sed_classes = np.where((variance > 1.00), 20, sed_classes)
    return sed_classes

def calculate_success_rate(confusion_matrix):
    diagonal = np.trace(confusion_matrix)
    diagonal_neighbors = np.sum(np.diagonal(confusion_matrix, offset=1)) + np.sum(np.diagonal(confusion_matrix, offset=-1))
    total_classified = np.sum(confusion_matrix)
    
    success_rate = (diagonal + diagonal_neighbors) / total_classified
    return success_rate

In [19]:
# Define the model architecture

initializer = tf.keras.initializers.GlorotNormal(seed = 1)
model = tf.keras.Sequential([
    layers.Dense(26, input_shape=[PCA_components], activation='sigmoid', kernel_initializer= initializer),
    layers.Dense(26, activation='sigmoid', kernel_initializer= initializer),
    layers.Dense(1, activation = 'linear', kernel_initializer= initializer)
])
model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = model_learning_rate)
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10, restore_best_weights=True)

start_time = time.time() # Measure execution time
model.fit(x_train, y_train, epochs= N_epochs, callbacks = [early_stopping], validation_data=(x_val,y_val)) # Train the model

# Calculate the execution time
end_time = time.time()
execution_time = end_time - start_time

C_pred = model.predict(x_test, verbose = 1).reshape(-1) # Predict the scalar parameter C using the committee    
SED_pred = CtoSEDarray(C_pred,np.zeros_like(C_pred))


Epoch 1/50
1/1 [==============================] - 1s 778ms/step - loss: 22.5677 - val_loss: 5.6559
Epoch 2/50
1/1 [==============================] - 0s 54ms/step - loss: 7.1273 - val_loss: 3.3965
Epoch 3/50
1/1 [==============================] - 0s 67ms/step - loss: 3.8660 - val_loss: 5.5817
Epoch 4/50
1/1 [==============================] - 0s 62ms/step - loss: 5.3838 - val_loss: 6.7829
Epoch 5/50
1/1 [==============================] - 0s 38ms/step - loss: 6.3890 - val_loss: 6.3534
Epoch 6/50
1/1 [==============================] - 0s 52ms/step - loss: 6.0248 - val_loss: 5.3447
Epoch 7/50
1/1 [==============================] - 0s 37ms/step - loss: 5.1910 - val_loss: 4.3454
Epoch 8/50
1/1 [==============================] - 0s 36ms/step - loss: 4.4167 - val_loss: 3.6345
Epoch 9/50
1/1 [==============================] - 0s 43ms/step - loss: 3.9526 - val_loss: 3.3298
Epoch 10/50
1/1 [==============================] - 0s 54ms/step - loss: 3.8868 - val_loss: 3.3657
Epoch 11/50
1/1 [==========

In [20]:
# Evaluate the performance of the classifier

mse = np.mean((y_test - y_pred)**2)
print('MSE:', mse)

f1 = f1_score(SED_test, SED_pred, average = None)
print('\nF1 score:', f1)
print(np.mean(f1[:13]))

for i in range(len(SED_test)):
    print(SED_test[i], C_pred[i], SED_pred[i])


confusion_matrix = tf.math.confusion_matrix(SED_test, SED_pred) 
print("\nConfusion matrix:")
print(confusion_matrix)

success_rate = calculate_success_rate(confusion_matrix)
print('\nSuccess rate:', success_rate)

print("Execution time:", execution_time, "seconds")


MSE: 2.462505780239917

F1 score: [0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.0
0 4.247594 5
11 4.247595 5
4 4.24759 5
7 4.2476025 5
6 4.247599 5
3 4.2475886 5
3 4.247602 5
4 4.2475963 5
9 4.247593 5
8 4.247597 5

Confusion matrix:
tf.Tensor(
[[0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]], shape=(12, 12), dtype=int32)

Success rate: 0.3
Execution time: 2.293637275695801 seconds


In [16]:
model.save('/Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/my_model/')

INFO:tensorflow:Assets written to: /Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/my_model/assets


INFO:tensorflow:Assets written to: /Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/my_model/assets
